In [ ]:
import numpy as np
import pandas as pd
import re
import requests
import holoviews as hv
import matplotlib.pyplot as plt
import pint
from scipy.stats import norm

In [ ]:
%load_ext autoreload
%autoreload 2
import simulation

In [ ]:
u = simulation.ureg

In [ ]:
hv.extension("bokeh")

# Photophysics

In [ ]:
power = 75 * u.milliwatts

In [ ]:
width = 26.5 * u.mm / 20
height = 0.213 * u.um

In [ ]:
(power / (width * height)).to("kW/cm^2")

# Femtosecond laser

In [ ]:
norm.pdf(521, 500, 4.4)

In [ ]:
norm.pdf(510, 500, 6.9 / 2)

In [ ]:
temporal_fwhm = 120 * u.femtoseconds
lmbda = 700 * u.nanometer
delta_lmbda = 40 * u.nanometer
xs = np.linspace(lmbda - delta_lmbda, lmbda + delta_lmbda, 10 * delta_lmbda.magnitude)
normalization = (len(xs) / (xs[-1] - xs[0])).magnitude
tbwp_quality = 1.5
tbwp_gaussian = 0.44 * tbwp_quality  # gaussian
tbwp_sech = 0.315 * tbwp_quality  # sech^2
# tbwp = 0.6 # Levante Emerald 2 ps

In [ ]:
spectral_width = (lmbda**2 / (u.speed_of_light * temporal_fwhm)).to("nm")
spectral_width

In [ ]:
spectral_width * tbwp_gaussian

In [ ]:
spectral_sigma = tbwp_gaussian * spectral_width / (2 * np.sqrt(2 * np.log(2)))
spectral_sigma

In [ ]:
gaussian_spectrum = norm.pdf(xs, lmbda, spectral_sigma)
gaussian_cdf = np.cumsum(gaussian_spectrum) / normalization

In [ ]:
hv.Curve((xs, np.log10(gaussian_spectrum))) * hv.Curve((xs, np.log(1 - gaussian_cdf)))

In [ ]:
sech_spectral_fwhm = tbwp_sech * spectral_fwhm

In [ ]:
sech_alpha = sech_spectral_fwhm / np.arccosh(np.sqrt(2))

In [ ]:
sech_spectrum = (
    1
    / (np.sqrt(sech_alpha) * np.cosh(2 * (xs - xs[xs.shape[0] // 2]) / sech_alpha)) ** 2
)
sech_cdf = np.cumsum(sech_spectrum.magnitude) / normalization

In [ ]:
hv.Curve((xs, np.log10(sech_spectrum.magnitude))) * hv.Curve((xs, np.log(1 - sech_cdf)))

In [ ]:
hv.Curve((xs, np.log10(gaussian_spectrum))) * hv.Curve(
    (xs, np.log10(sech_spectrum.magnitude))
)

# Laser + filter

In [ ]:
# filters, filter_peaks = load_filter_spectra("LF102474.csv.csv")
filters, filter_peaks = simulation.load_filter_spectra("LF103454.csv")

In [ ]:
sorted(filter_peaks.items())

In [ ]:
hv.Curve(filter_spectrum)

In [ ]:
idx = 86.0
filter_spectrum = filters[idx]
# index = pd.Series(np.linspace(laser_cwl-40,laser_cwl+40,200))
# index = pd.Series(np.linspace(filter_spectrum.index[0],filter_spectrum.index[-1],10000))
# filter_spectrum = filter_spectrum.reindex(index=index)
# filter_spectrum.interpolate(method='nearest', inplace=True)
# index = filter_spectrum.index
filter_spectrum /= filter_spectrum.max()
filter_thresh = 0.8
laser_cwl = (filter_spectrum > filter_thresh)[::-1].idxmax()[0]
new_index = pd.Index(np.linspace(laser_cwl - 40, laser_cwl + 40, 100))
index = filter_spectrum.index.astype(np.float).union(new_index)
index.name = "wavelength"
filter_spectrum = filter_spectrum.reindex(index=index)
filter_spectrum.interpolate(method="linear", inplace=True)
index = filter_spectrum.index  # probably unnecessary
# laser_cwl = 730
# temporal_fwhm = 110 * u.femtoseconds
# spectral_width = (lmbda**2/(u.speed_of_light * temporal_fwhm)).to("nm")
# tbwp = 0.6
# laser_sigma = tbwp * spectral_width / (2 * np.sqrt(2 * np.log(2)))
laser_bandwidth = (
    200 * 1 / u.cm
)  # (tbwp/(u.speed_of_light * temporal_fwhm)).to("cm^-1")
laser_fwhm = (laser_bandwidth * (laser_cwl * u.nm) ** 2).to("nm")
laser_sigma = laser_fwhm / (2 * np.sqrt(2 * np.log(2)))
# laser_sigma = 2#5.1
laser_spectrum = norm.pdf(index.values, laser_cwl, laser_sigma)
laser_spectrum /= np.nanmax(laser_spectrum)
laser_spectrum = pd.Series(laser_spectrum, index=index, name="transmission")
# laser_spectrum[laser_spectrum == 0] = np.nan
output_spectrum = filter_spectrum.multiply(laser_spectrum, axis=0)
lines = (
    hv.VLine(x=laser_cwl).opts(color="red")
    * hv.VLine(x=laser_cwl + 5).opts(color="red", line_dash=[2, 8])
    * hv.VLine(x=laser_cwl + 10).opts(color="red", line_dash="dashed")
    * hv.VLine(x=laser_cwl + 15).opts(color="red", line_dash="dotdash")
    * hv.VLine(x=laser_cwl + 20).opts(color="red", line_dash="dotted")
)
(
    (
        hv.Curve(np.log10(laser_spectrum))
        * hv.Curve(np.log10(filter_spectrum))
        * hv.Curve(np.log10(output_spectrum)).opts(color="purple")
        * lines
        * hv.HLine(y=-4).opts(color="orange")
        * hv.HLine(y=-6).opts(color="orange")
    )
    .redim.range(wavelength=(laser_cwl - 10, laser_cwl + 25), transmission=(-8, 0))
    .opts(width=600)
    + (
        hv.Curve(laser_spectrum)
        * hv.Curve(filter_spectrum)
        * hv.Curve(output_spectrum).opts(color="purple")
        * lines
    )
    .redim(transmission=hv.Dimension("power", range=(0, 1)))
    .opts(width=600)
).cols(1)

In [ ]:
laser_fwhm

In [ ]:
hv.Curve(filters[idx])

In [ ]:
hv.Curve(filters[5.0])

In [ ]:
pd.read_csv("discovery_660nm.csv")

In [ ]:
beam_spectrum = pd.read_csv("discovery_660nm.csv").set_index("Wavelength")
beam_spectrum /= beam_spectrum.max()
laser_cwl = beam_spectrum.idxmax()[0]
log_beam_spectrum = np.log10(beam_spectrum)
log_beam_spectrum[np.isinf(log_beam_spectrum)] = np.nan

In [ ]:
(
    hv.Curve(log_beam_spectrum).redim.range(
        wavelength=(laser_cwl - 10, laser_cwl + 25), transmission=(-8, 0)
    )
    * hv.VLine(x=laser_cwl).opts(color="red")
    * hv.VLine(x=laser_cwl + 5).opts(color="red", line_dash=[2, 8])
    * hv.VLine(x=laser_cwl + 10).opts(color="red", line_dash="dashed")
    * hv.VLine(x=laser_cwl + 15).opts(color="red", line_dash="dotdash")
    * hv.VLine(x=laser_cwl + 20).opts(color="red", line_dash="dotted")
    * hv.HLine(y=-1).opts(color="orange")
    * hv.HLine(y=-2).opts(color="orange")
    * hv.HLine(y=-3).opts(color="orange")
).opts(width=600)

# Spectral response

In [ ]:
def bin_spectrum(df, bins):
    bin_assignment = pd.cut(df.index, bins).rename_categories(
        (bins.right + bins.left) / 2
    )
    return df.groupby(bin_assignment).mean()

In [ ]:
spectra_urls = {
    "mCherry": "https://www.fpbase.org/spectra_csv/?q=79,80,158",
    "mCherry2": "https://www.fpbase.org/spectra_csv/?q=1451,1450",
}
spectra = {
    name: pd.read_csv(url)
    .rename(columns={f"{name} {kind}": kind for kind in ("ex", "em", "2p")})
    .set_index("wavelength")
    for name, url in spectra_urls.items()
}

In [ ]:
bins = pd.interval_range(300, 1100, freq=5)

In [ ]:
binned_spectra = {
    name: bin_spectrum(spectrum, bins) for name, spectrum in spectra.items()
}

In [ ]:
(
    hv.Curve(spectra["mCherry"]["em"].dropna())
    * hv.Curve(spectra["mCherry2"]["em"].dropna())
).opts(width=800)

In [ ]:
(
    hv.Curve(binned_spectra["mCherry"]["em"].dropna())
    * hv.Curve(binned_spectra["mCherry2"]["em"].dropna())
).opts(width=800)

In [ ]:
obj_transmission = pd.read_csv("uplxapo_20x.csv", header=None)
obj_transmission.columns = ["wavelength", "transmission"]
obj_transmission.set_index("wavelength", inplace=True)
hv.Curve(obj_transmission).opts(width=800)

In [ ]:
laser_power_shg = pd.read_csv("discovery_vue_shg.csv", header=1).set_index("lambda")
# laser_power_shg.columns = ["power"]
laser_power_pump = pd.read_csv("discovery_vue_pump.csv", header=1).set_index("lambda")
laser_power_pump.columns = ["pump power"]
laser_power = pd.concat(
    [
        laser_power_shg,
        laser_power_pump,
        laser_power_shg.rename(columns={"SHG power": "power"}).append(
            laser_power_pump.rename(columns={"pump power": "power"})
        ),
    ],
    axis=1,
)
laser_power.index.name = "wavelength"
hv.Curve(laser_power, "wavelength", "power").opts(width=800)

In [ ]:
laser_power

# Image data

In [ ]:
nd2 = nd2reader.ND2Reader(
    "/home/jqs1/scratch/191312/G-R_RG/RG_100pct_100ms_100pct_100ms.nd2_0001.nd2"
)

In [ ]:
plt.imshow(nd2.get_frame_2D(t=0))

In [ ]:
import pickle

In [ ]:
with open(
    "/n/groups/paulsson/jqs1/molecule-counting/190521photobleaching_noflatcorr.pickle",
    "rb",
) as f:
    dat = pickle.load(f)

In [ ]:
dat.keys()

In [ ]:
f = np.asarray(
    dat["/n/scratch2/jqs1/190514/GFP_100ms_10pct.nd2"][0]["segmentation_frame"]
)

In [ ]:
plt.imshow(f)